In [2]:
import os
from collections import defaultdict


labels_path = "/kaggle/input/gemstone-v3/dataset/labels"  

class_counts = defaultdict(int)
image_object_counts = []
empty_files = 0
total_objects = 0

for file in os.listdir(labels_path):
    if not file.endswith(".txt"):
        continue
        
    file_path = os.path.join(labels_path, file)
    
    with open(file_path, "r") as f:
        lines = f.readlines()
        
      
        if len(lines) == 0:
            empty_files += 1
        
      
        num_objects = len(lines)
        image_object_counts.append(num_objects)
        total_objects += num_objects
        
  
        for line in lines:
            cls = int(line.split()[0])
            class_counts[cls] += 1


print("📊 DATASET SUMMARY")
print("="*40)
print(f"Total label files (images): {len(image_object_counts)}")
print(f"Total objects: {total_objects}")
print(f"Empty label files: {empty_files}")

print("\n📦 Objects per class:")
for cls, count in sorted(class_counts.items()):
    print(f"Class {cls}: {count}")

print("\n🖼️ Objects per image:")
print(f"Min: {min(image_object_counts)}")
print(f"Max: {max(image_object_counts)}")
print(f"Average: {sum(image_object_counts)/len(image_object_counts):.2f}")


📊 DATASET SUMMARY
Total label files (images): 469
Total objects: 1074
Empty label files: 0

📦 Objects per class:
Class 0: 224
Class 1: 123
Class 2: 146
Class 3: 162
Class 4: 144
Class 5: 80
Class 6: 84
Class 7: 111

🖼️ Objects per image:
Min: 1
Max: 27
Average: 2.29


In [12]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:0000:01


In [8]:
import os
import shutil
from collections import defaultdict


images_path = "/kaggle/input/gemstone-v3/dataset/images"
labels_path = "/kaggle/input/gemstone-v3/dataset/labels"
output_path = "/kaggle/working/split_dataset_final"


for split in ["train", "val"]:
    os.makedirs(f"{output_path}/images/{split}", exist_ok=True)
    os.makedirs(f"{output_path}/labels/{split}", exist_ok=True)


class_files = defaultdict(list)

for file in os.listdir(labels_path):
    if not file.endswith(".txt"):
        continue

    with open(os.path.join(labels_path, file)) as f:
        classes = set(int(line.split()[0]) for line in f.readlines())

    for cls in classes:
        class_files[cls].append(file)


train_ratio = 0.8

train_files = set()
val_files = set()


for cls, files in class_files.items():
    files = list(set(files)) 
    files.sort()  

    split_idx = int(len(files) * train_ratio)

    train_files.update(files[:split_idx])
    val_files.update(files[split_idx:])


val_files -= train_files


def copy_files(file_set, split):
    for file in file_set:
      
        shutil.copy(
            os.path.join(labels_path, file),
            os.path.join(output_path, "labels", split, file)
        )

      
        base = file.replace(".txt", "")
        found = False

        for ext in [".jpg", ".png", ".jpeg", ".JPG"]:
            img_path = os.path.join(images_path, base + ext)
            if os.path.exists(img_path):
                shutil.copy(
                    img_path,
                    os.path.join(output_path, "images", split, base + ext)
                )
                found = True
                break

        if not found:
            print(f"⚠️ Missing image: {file}")


copy_files(train_files, "train")
copy_files(val_files, "val")

print("✅ Train–Validation split completed!")

print(f"\nTrain images: {len(train_files)}")
print(f"Validation images: {len(val_files)}")


✅ Train–Validation split completed!

Train images: 371
Validation images: 98


In [9]:
import os
from collections import defaultdict

base_path = "/kaggle/working/split_dataset_final/labels"

splits = ["train", "val"]

for split in splits:
    split_path = os.path.join(base_path, split)
    
    class_counts = defaultdict(int)
    total_objects = 0
    image_count = 0

    for file in os.listdir(split_path):
        if not file.endswith(".txt"):
            continue

        image_count += 1

        with open(os.path.join(split_path, file), "r") as f:
            lines = f.readlines()

            for line in lines:
                cls = int(line.split()[0])
                class_counts[cls] += 1
                total_objects += 1

    print(f"\n📊 {split.upper()} SPLIT")
    print("="*35)
    print(f"Images: {image_count}")
    print(f"Total objects: {total_objects}")

    for cls in sorted(class_counts.keys()):
        print(f"Class {cls}: {class_counts[cls]}")



📊 TRAIN SPLIT
Images: 371
Total objects: 815
Class 0: 172
Class 1: 88
Class 2: 102
Class 3: 125
Class 4: 110
Class 5: 63
Class 6: 69
Class 7: 86

📊 VAL SPLIT
Images: 98
Total objects: 259
Class 0: 52
Class 1: 35
Class 2: 44
Class 3: 37
Class 4: 34
Class 5: 17
Class 6: 15
Class 7: 25


In [10]:
data_yaml = """
path: /kaggle/working/split_dataset_final

train: images/train
val: images/val

names:
  0: Tigers_Eye
  1: Obsidian
  2: Lapis_Lazuli
  3: Rose_Quartz
  4: Red_Jasper
  5: Clear_Quartz
  6: Amethyst
  7: Aventurine
"""

with open("/kaggle/working/data.yaml", "w") as f:
    f.write(data_yaml)

print("✅ data.yaml created!")


✅ data.yaml created!


In [13]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

model.train(
    data="/kaggle/working/data.yaml",
    
   
    epochs=150,
    imgsz=832,
    batch=16,
    patience=30,

   
    optimizer="AdamW",
    lr0=0.001,

    
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2,
    
    fliplr=0.5,
    flipud=0.1,
    
    mosaic=1.0,
    mixup=0.1,
    
  
    weight_decay=0.0005,
    
   
    plots=True,
    save=True
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=832, int8=Fa

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b9458be7800>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [27]:
import shutil
import os

base_path = "/kaggle/working/runs/detect"
output_path = "/kaggle/working/models"

os.makedirs(output_path, exist_ok=True)


for folder in os.listdir(base_path):
    model_path = os.path.join(base_path, folder, "weights", "best.pt")
    
    if os.path.exists(model_path):
        new_name = f"{folder}_best.pt"
        shutil.copy(model_path, os.path.join(output_path, new_name))

print("✅ All models copied to /kaggle/working/models")


✅ All models copied to /kaggle/working/models


In [26]:
import shutil

shutil.make_archive(
    "/kaggle/working/models_backup",
    'zip',
    "/kaggle/working/models"
)


'/kaggle/working/models_backup.zip'